# Notebook code was buggy 👾, I was able to identify the coding issues 👽 and replaced some lines of code for full functionality. 🤖
# Any ideas for running a comparison between submission_predict_1 and submission_predict_2 would be appreciated.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.gridspec as gridspec
from termcolor import colored
import gc,time,os


plt.style.use('seaborn-whitegrid')
import warnings
warnings.simplefilter("ignore")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

1. train.csv -> showing the price of the item and the number of items sold on each date.

2. shop.csv -> Details of shops corresponding to the shop id in train.csv.

3. item.csv -> Details of items corresponding to item id in train.csv

4. item_categories.csv -> Details of item category corresponding to category ID in item.csv

5. test.csv -> test data for prediction

train.csv
--------

- date: date in format dd/mm/yy.
- date_block_num: a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1 and so on.
- shop_id: unique identifier of a shop.
- item_id: unique identifier of a product.
- item_price: current price of an item.
- item_cnt_day: number of products sold. We are predicting a monthly amount of this measure.


shop.csv
--------
- shop_name: shop name corresponding to shop id in train.csv
- shop_id

item.csv
--------
- item_name: item name corresponding to item id in train.csv
- item_id
- item_category_id: category id of item


item_category.csv
-----------------
- item_category_name: category name of item cooresponding to item_category_id in item.csv 
- category_id

test.csv
----------
- ID - an Id that represents a (Shop, Item) tuple within the test set
- shop_id
- item_id

In [ ]:
TRAIN_SALES_CSV = '../input/english-converted-datasets/sales_train.csv'
SHOPS_CSV = '../input/english-converted-datasets/shops.csv'
ITEMS_CSV= '../input/english-converted-datasets/items.csv'
ITEM_CATEGORY_CSV = '../input/english-converted-datasets/item_categories.csv'
TEST_CSV ='../input/english-converted-datasets/test.csv'

In [ ]:
def show_data(df_train,heading='TRAIN DATA'):
    
    """
    function which gives basic data information
    Args:
        df_train: pandas dataframe
        heading: deading to display
    Returns:
        None
    """

    print(colored(heading,'red'))
    print('')
    print('Date shape')
    print(f'shape:{df_train.shape}')
    print('')
    print('--'*50)
    print('')
    print('Sample:')
    print(df_train.head(3).to_markdown())
    print('')
    print('--'*50)
    print('')
    print('Columns and data types:')
    print('')
    print(df_train.info())

In [ ]:
df_train = pd.read_csv(TRAIN_SALES_CSV)
df_shop = pd.read_csv(SHOPS_CSV)
df_item = pd.read_csv(ITEMS_CSV)
df_itemcat = pd.read_csv(ITEM_CATEGORY_CSV)

# include some fancy print formatting 🤓

show_data(df_train,heading='TRAIN DATA')
print('')
print('__'*40)
print('')
show_data(df_shop,heading='SHOP DATA')
print('')
print('__'*40)
print('')
show_data(df_item,heading='ITEM DETAILS DATA')
print('__'*40)
print('')
show_data(df_itemcat,heading='ITEM CATEGORY DATA')

In [ ]:
df_test = pd.read_csv(TEST_CSV)
show_data(df_test,heading='TEST DATA')

**Observation:** 
As mentioned before for test data, we only have shop id and item id. The train data is spread over 4 files. From the examples above, we know that there are common identifiers across multiple files. It will be great moving forward once we merge these train files based on a common ID.

In [ ]:
df_test = pd.read_csv(TEST_CSV)
show_data(df_test,heading='TEST DATA')

In [ ]:
### Merging all dataframes together
dff = df_train.merge(df_item,on="item_id")  # <-- this is a code fix so the notebook can properly function
#dff = dff.merge(df_itemcat,on="item_category_id")  # doesn't work properly removed
dff = dff.merge(df_shop,on="shop_id")
dff = dff.drop(columns=["item_name"])
dff.to_csv('merged_original.csv',index=False)

In [ ]:
df = dff.copy()
# df.to_csv('merged_original.csv',index=False)
df.head()

# Statistical Description of Dataframe

In [ ]:
df.describe()


# Timeframe of Dataset

In [ ]:
df["date"]=  pd.to_datetime(df["date"], format='%d.%m.%Y')
df.sort_values(by="date", ascending=True, inplace=True)
print(f'Minimum data present: {df["date"].min()}')
print(f'Maximum date present: {df["date"].max()}')

In [ ]:
# sorting dataframe based on date
df = df.sort_values(by='date').reset_index(drop=True)
df.head()

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(15,5))
sns.boxplot(df['item_cnt_day'],ax=axes[0])
axes[0].set_title('Boxplot')
sns.distplot(df['item_cnt_day'],ax=axes[1])
axes[1].set_title('Distribution')
plt.suptitle('No of units sold(Item Cnt day)',fontsize="20")
plt.show()

In [ ]:
df['item_cnt_day'].describe()


In [ ]:
print(f'Minimum value {df["item_cnt_day"].min()}')
print(f'Maximum value {df["item_cnt_day"].max()}')

In [ ]:
# Let us print percentile values
for i in range(0,101,10):
    print(f'{i}th percentile value for item_cnt_day is {np.percentile(df["item_cnt_day"],i)}')
    
print('--'*50)

for i in range(90,100):
    print(f'{i}th percentile value for item_cnt_day is {np.percentile(df["item_cnt_day"],i)}')
    
print('--'*50)

for i in range(1,10):
    k = 99 + i/10 
    print(f'{k}th percentile value for item_cnt_day is {np.percentile(df["item_cnt_day"],k)}')

In [ ]:
# we will remove some extreme outliers
df[df['item_cnt_day'] > df['item_cnt_day'].quantile(0.95)]

# Identify less than zero item count perday, will be removed

In [ ]:
df[df['item_cnt_day'] < 0]


In [ ]:
print(f'shape of data before outliers: {df.shape}')
df = df[df['item_cnt_day'] >= 0]
upper_quantile = df['item_cnt_day'].quantile(0.95)
print(f'Removing values greater that upper_quantile {upper_quantile} and less than 0')
df['item_cnt_day'] = np.where(df['item_cnt_day'] > upper_quantile, upper_quantile, df['item_cnt_day'])

print(f'shape of data after removing outliers: {df.shape}')
print(f'Minimum units of product sold a time {df["item_cnt_day"].min()}')
print(f'Maximum units of product sold a time {df["item_cnt_day"].max()}')

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(15,5))
sns.boxplot(df['item_price'],ax=axes[0])
axes[0].set_title('Boxplot')
sns.distplot(df['item_price'],ax=axes[1])
axes[1].set_title('Distribution')
plt.suptitle('Item Price per unit',fontsize="20")
plt.show()


In [ ]:
df['item_price'].describe()


In [ ]:
# Let us print percentile values
for i in range(0,101,10):
    print(f'{i}th percentile value for item_price is {np.percentile(df["item_price"],i)}')
    
print('--'*50)

for i in range(90,100):
    print(f'{i}th percentile value for item_price is {np.percentile(df["item_price"],i)}')
    
print('--'*50)

for i in range(1,10):
    k = 99 + i/10 
    print(f'{k}th percentile value for item_price is {np.percentile(df["item_price"],k)}')

# Removing Outliers

In [ ]:
# we have an extreme outlier value in item price. Let us remove it

print(f'shape of data before removing outliers: {df.shape}')
df = df[df['item_price'] >= 0]
upper_quantile = df['item_price'].quantile(0.95)
df['item_price'] = np.where(df['item_price'] > upper_quantile, upper_quantile, df['item_price'])
print(f'shape of data after removing outliers: {df.shape}')

print(f'Minimum price of a single item {df["item_price"].min()}')
print(f'Maximum price ofa single item {df["item_price"].max()}')

In [ ]:
df_tmp = df[df['shop_id'] == 31][['date','item_id','item_price']].reset_index(drop=True)
items = df_tmp['item_id'].unique()[0:8]

fig,axes = plt.subplots(1,1,figsize=(25,8))
colors = ['red','orange','blue','green','yellow','purple','cyan','brown']
for i,item in enumerate(items):
    dprice = df_tmp[df_tmp['item_id'] == item][['item_price','date']]
    
    sns.lineplot(x=dprice['date'],y=dprice['item_price'],ax=axes,color=colors[i],label=item)
    
axes.set_title('Price change of items - shop_id 31',fontsize="28")
axes.legend()
plt.show()


df_tmp = df[df['shop_id'] == 28][['date','item_id','item_price']].reset_index(drop=True)
items = df_tmp['item_id'].unique()[0:8]

fig,axes = plt.subplots(1,1,figsize=(25,8))
colors = ['red','orange','blue','green','yellow','purple','cyan','brown']
for i,item in enumerate(items):
    dprice = df_tmp[df_tmp['item_id'] == item][['item_price','date']]
    
    sns.lineplot(x=dprice['date'],y=dprice['item_price'],ax=axes,color=colors[i],label=item)
    
axes.set_title('Price change of items - shop_id 28',fontsize="28")
axes.legend()
plt.show()



df_tmp = df[df['shop_id'] == 21][['date','item_id','item_price']].reset_index(drop=True)
items = df_tmp['item_id'].unique()[0:8]

fig,axes = plt.subplots(1,1,figsize=(25,8))
colors = ['red','orange','blue','green','yellow','purple','cyan','brown']
for i,item in enumerate(items):
    dprice = df_tmp[df_tmp['item_id'] == item][['item_price','date']]
    
    sns.lineplot(x=dprice['date'],y=dprice['item_price'],ax=axes,color=colors[i],label=item)
    
axes.set_title('Price change of items - shop_id 21',fontsize="28")
axes.legend()
plt.show()

In [ ]:
df['Sales_per_item'] = df['item_cnt_day'] * df['item_price']



In [ ]:
fig = plt.figure(figsize=(25,7))
gs = fig.add_gridspec(1, 3)
ax00 = fig.add_subplot(gs[0,0])
ax01 = fig.add_subplot(gs[0,1])
ax02 = fig.add_subplot(gs[0,2])

# setting size of xlabel and ylabel

ax00.tick_params(axis='both', labelsize=15)
ax01.tick_params(axis='both', labelsize=15)
ax02.tick_params(axis='both', labelsize=15)
ax00.set_title('Number of sales per item', fontsize=20)
ax01.set_title('Item price distribution', fontsize=20)
ax02.set_title('Item count distribution', fontsize=20)
sns.histplot(data = df ,x="Sales_per_item", kde=True, bins=50,ax=ax00, color="violet")
sns.histplot(data = df ,x="item_price", kde=True, ax=ax01, bins=50, color="tomato")
sns.histplot(data = df ,x="item_cnt_day", kde=False, ax=ax02, bins=20, color="cornflowerblue")

fig.subplots_adjust(top=0.8)
fig.suptitle('Sales Feature Distributions per Day', fontsize="28");

In [ ]:
train_shop_ids = set(df['shop_id'].unique())
test_shop_ids = set(df_test['shop_id'].unique())

train_item_ids = set(df['item_id'].unique())
test_item_ids = set(df_test['item_id'].unique())

print(f'There are about {len(train_shop_ids)} unique shop ids in train data and {len(test_shop_ids)} shop ids in test data')
print(f'There are about {len(train_item_ids)} unique item ids in train data and {len(test_item_ids)} item ids in test data')
print('--'*50)

df['pair'] = df[['shop_id','item_id']].apply(lambda x: str(x['shop_id'])+'_'+str(x['item_id']),axis=1)
df_test['pair'] = df_test[['shop_id','item_id']].apply(lambda x: str(x['shop_id'])+'_'+str(x['item_id']),axis=1)
train_pair_ids = set(df['pair'].unique())
test_pair_ids = set(df_test['pair'].unique())

print(f'There are {len(train_shop_ids - test_shop_ids)} shop ids present in train data which are not in test data')
print(f'There are {len(train_item_ids - test_item_ids)} item ids present in train data which are not in test data')
print(f'There are {len(train_pair_ids - test_pair_ids)} shop id item id pairs present in train data which are not in test data')

print('--'*50)

print(f'There are {len(test_item_ids - train_item_ids)} item ids present in test data which are not in train data')
print(f'There are {len(test_shop_ids - train_shop_ids)} shop ids present in test data which are not in train data')
print(f'There are {len(test_pair_ids - train_pair_ids)} shop id item id pairs present in test data which are not in train data')

In [ ]:
print(f'Total number of unique shop ids: {df["shop_id"].nunique()}')
df_tmp = df[["shop_id","Sales_per_item","item_cnt_day"]]
df_tmp= pd.pivot_table(data=df_tmp,index=["shop_id"],aggfunc={"item_cnt_day":np.sum,"Sales_per_item":np.sum}).reset_index()


fig, axes = plt.subplots(2,1,figsize=(20,10))
sns.barplot(x=df_tmp["shop_id"],y=df_tmp["item_cnt_day"],ax=axes[0])
axes[0].set_title("Total number of units sold among various shops")
sns.barplot(x=df_tmp["shop_id"],y=df_tmp["Sales_per_item"],ax=axes[1])
axes[1].set_title('Total turn over in various shops')
plt.suptitle('Shop id', fontsize="28")
plt.show()

In [ ]:
print(f'Total number of unique item ids: {df["item_id"].nunique()}')

df_tmp = df[["item_id","Sales_per_item","item_cnt_day"]]
df_tmp= pd.pivot_table(data=df_tmp,index=["item_id"],aggfunc={"item_cnt_day":np.sum,"Sales_per_item":np.sum}).reset_index()
df_tmp_sales = df_tmp.sort_values(by=['Sales_per_item'],ascending=False).head(50).reset_index(drop=True)
df_tmp_count = df_tmp.sort_values(by=['item_cnt_day'],ascending=False).head(50).reset_index(drop=True)

fig, axes = plt.subplots(2,1,figsize=(20,15))
sns.barplot(x=df_tmp_count["item_id"],y=df_tmp_count["item_cnt_day"],ax=axes[0])
axes[0].set_title("Top selling items of no of units sold")
axes[0].set_xticklabels(axes[0].get_xticklabels(),rotation=45)
sns.barplot(x=df_tmp_sales["item_id"],y=df_tmp_sales["Sales_per_item"],ax=axes[1])
axes[1].set_title('Top selling items in terms of Turn over')
axes[1].set_xticklabels(axes[1].get_xticklabels(),rotation=45)
plt.suptitle('Item id', fontsize="28")
plt.show()
# del df_tmp,df_tmp_count,df_tmp_sales

In [ ]:
#print(f'Total number of unique item categorical ids: {df["item_category_id"].nunique()}')  # doesn't work properly, commented out
print(f'Total number of unique item categorical ids: {df["category_id"].nunique()}')  # <-- this is a code fix so the notebook can properly function

#df_tmp = df[["item_category_id","Sales_per_item","item_cnt_day"]]  df_tmp = df[["category_id","Sales_per_item","item_cnt_day"]]
df_tmp = df[["category_id","Sales_per_item","item_cnt_day"]]   # <-- this is a code fix so the notebook can properly function

#df_tmp= pd.pivot_table(data=df_tmp,index=["item_category_id"],aggfunc={"item_cnt_day":np.sum,"Sales_per_item":np.sum}).reset_index()  # doesn't work properly, commented out
df_tmp= pd.pivot_table(data=df_tmp,index=["category_id"],aggfunc={"item_cnt_day":np.sum,"Sales_per_item":np.sum}).reset_index()  # <-- this is a code fix so the notebook can properly function
df_tmp_sales = df_tmp.sort_values(by=['Sales_per_item'],ascending=False).head(50).reset_index(drop=True)
df_tmp_count = df_tmp.sort_values(by=['item_cnt_day'],ascending=False).head(50).reset_index(drop=True)

fig, axes = plt.subplots(2,1,figsize=(20,15))
#sns.barplot(x=df_tmp_count["item_category_id"],y=df_tmp_count["item_cnt_day"],ax=axes[0])  # doesn't work properly, commented out
sns.barplot(x=df_tmp_count["category_id"],y=df_tmp_count["item_cnt_day"],ax=axes[0])  # <-- this is a code fix so the notebook can properly function
axes[0].set_title("Top selling items of no of units sold")
axes[0].set_xticklabels(axes[0].get_xticklabels(),rotation=45)
#sns.barplot(x=df_tmp_sales["item_category_id"],y=df_tmp_sales["Sales_per_item"],ax=axes[1])  # doesn't work properly, commented out
sns.barplot(x=df_tmp_sales["category_id"],y=df_tmp_sales["Sales_per_item"],ax=axes[1])   # <-- this is a code fix so the notebook can properly function
axes[1].set_title('Top selling items in terms of Turn over')
axes[1].set_xticklabels(axes[1].get_xticklabels(),rotation=45)
plt.suptitle('Item Categorical id', fontsize="28")
plt.show()
del df_tmp,df_tmp_count,df_tmp_sales

In [ ]:
#item_categories = df['item_category_id'].unique()   # doesn't work properly, commented out
item_categories = df['category_id'].unique()   # <-- this is a code fix so the notebook can properly function

#tmp = df[['item_id','item_category_id']].groupby(by="item_id").nunique().reset_index()
tmp = df[['item_id','category_id']].groupby(by="item_id").nunique().reset_index()
tmp.head()

In [ ]:
df["Year"] = df["date"].dt.year
df["Month"] = df["date"].dt.month
df["day_of_month"] = df["date"].dt.day
df["day_of_week"] = df["date"].dt.day_of_week

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(25,12))
df_tmp = df[['date_block_num','Month','Sales_per_item']].groupby(by=['date_block_num']).aggregate("sum").reset_index()
sns.lineplot(x=df_tmp['date_block_num'],y=df_tmp['Sales_per_item'],ax=axes[0])
axes[0].set_title('Total turn over (Total Sales)',fontsize="25")
axes[0].set_xlabel('Date',fontsize="20")
axes[0].set_ylabel('Turn over per month',fontsize="20")


df_tmp = df[['date_block_num','Month','item_cnt_day']].groupby(by=['date_block_num']).aggregate("sum").reset_index()
sns.lineplot(x=df_tmp['date_block_num'],y=df_tmp['item_cnt_day'],ax=axes[1])
axes[1].set_title('Total units sold',fontsize="25")
axes[1].set_xlabel('Date',fontsize="20")
axes[1].set_ylabel('Turn over per month',fontsize="20")

plt.tight_layout()
del df_tmp
plt.show()

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(25,8))
df_tmp = df[['Year','Month','Sales_per_item']].pivot_table(index=['Month'],columns=['Year'],aggfunc={"Sales_per_item":np.sum})
axes[0].plot(df_tmp)
axes[0].set_title('Total turn over (Total Sales)')
axes[0].legend(labels=[i[1] for i in df_tmp.columns])

df_tmp = df[['Year','Month','item_cnt_day']].pivot_table(index=['Month'],columns=['Year'],aggfunc={"item_cnt_day":np.sum})
axes[1].plot(df_tmp)
axes[1].set_title('Total no of units sold')
axes[1].legend(labels=[i[1] for i in df_tmp.columns])
plt.suptitle('Monthly Sales - Yearly',fontsize="28")
plt.show()


In [ ]:
df_tmp = df[['date_block_num','shop_id','item_cnt_day']]
df_tmp.groupby(by='date_block_num').aggregate({'shop_id':'nunique'}).reset_index()

In [ ]:
df_tmp = df[['date_block_num','shop_id','item_cnt_day']]
dt = pd.pivot_table(index='date_block_num',data=df_tmp,columns='shop_id',aggfunc="sum").reset_index(drop=True)
dt = dt.item_cnt_day
dt.columns.name = 'Month'
dt

In [ ]:
def remove_outliers(df):
    #remove outliers from item_cnt_day
    df = df[df['item_cnt_day'] >= 0]
    upper_quantile = df['item_cnt_day'].quantile(0.95)
    df['item_cnt_day'] = np.where(df['item_cnt_day'] > upper_quantile, upper_quantile, df['item_cnt_day'])
    
    df = df[df['item_price'] >= 0]
    upper_quantile = df['item_price'].quantile(0.95)
    df['item_price'] = np.where(df['item_price'] > upper_quantile, upper_quantile, df['item_price'])
    return df

In [ ]:
#load the merged data
df_train = pd.read_csv('merged_original.csv')
df_test = pd.read_csv('../input/english-converted-datasets/test.csv') # <-- this is a code fix so the notebook can properly function
print(df_train.shape,df_test.shape)


In [ ]:
#removing outliers as per our analysis
df_train = remove_outliers(df_train)
df_train.shape

In [ ]:
df_train['item_cnt_day'] = df_train['item_cnt_day'].clip(0,20)

In [ ]:
from itertools import product
ts = time.time()
matrix = []
cols = ['date_block_num','shop_id','item_id']

for i in range(34):
    
    # for each month, we are appending all month,shop_id,item_id combinations
    sales = df_train[df_train['date_block_num']==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
        
# creating a dataframe from list
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
# convert to np.int8 to reduce memory usage
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)
matrix = matrix.reset_index(drop=True)
print(f'{time.time() - ts}')

In [ ]:
print(matrix.shape)
matrix.head()

In [ ]:
# df_test = df_test.drop(columns=['ID'])
# assigning month variable
df_test['date_block_num'] = 34

# conacting with train matrix
matrix = pd.concat([matrix,df_test],axis=0).reset_index(drop=True)

matrix.shape

In [ ]:
# example
df_train.loc[(df_train['date_block_num'] == 0) &  (df_train['shop_id'] == 0) & (df_train['item_id'] == 32)]

In [ ]:
tmp = df_train[['date_block_num','shop_id','item_id','item_cnt_day']]
tmp = tmp.groupby(by=['date_block_num','shop_id','item_id']).agg({'item_cnt_day':'sum'})
tmp.columns = ['item_cnt_month']
tmp = tmp.reset_index(drop=False)
tmp.head()

In [ ]:
#merging with main frame
matrix = matrix.merge(tmp,on=['date_block_num','shop_id','item_id'],how='left')
matrix = matrix.fillna(0)

#clip values of max items sold per month to 20
matrix['item_cnt_month'] = matrix['item_cnt_month'].clip(0,20)
del tmp

In [ ]:
matrix.head()


In [ ]:
def lag_feature(df, lags, col):
    """
    creates a lag feature. eg: item_cnt_month corresponding to a shop_id item_id combination of previous month, 
    previous second month etc
    """
    
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df


In [ ]:
ts = time.time()
matrix = lag_feature(matrix, [1,2,3,4,5,6,7,8,9,10,11,12], 'item_cnt_month')
#fill missing values
matrix = matrix.fillna(0)
time.time() - ts

In [ ]:
feats = [i for i in matrix.columns if 'lag' in i] + ['shop_id','item_id','date_block_num','item_cnt_month']

data = matrix[feats]

train_df = data.loc[(data['date_block_num'] > 11) & (data['date_block_num'] < 33)]
val_df = data.loc[data['date_block_num'] == 33]
test_df = data.loc[data['date_block_num'] == 34]

X_train = train_df.drop(columns=['shop_id','item_id','date_block_num','item_cnt_month'])
y_train = train_df['item_cnt_month']
X_val = val_df.drop(columns=['shop_id','item_id','date_block_num','item_cnt_month'])
y_val = val_df['item_cnt_month']
X_test = test_df.drop(columns=['shop_id','item_id','date_block_num','item_cnt_month'])

print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)

In [ ]:
del data
del train_df
del val_df
del test_df

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# First RandomForestRegressor Model
model = RandomForestRegressor(random_state=42,max_depth=5,n_estimators=50,min_samples_split=4)
model.fit(X_train,y_train)
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

train_rmse = mean_squared_error(y_train,y_train_pred,squared=False)
val_rmse = mean_squared_error(y_val,y_val_pred,squared=False)

print(f'Train rmse: {train_rmse}')
print(f'Val rmse: {val_rmse}')

In [ ]:
# Second RandomForestRegressor Model
model2 = RandomForestRegressor(random_state=65,max_depth=8,n_estimators=80,min_samples_split=8)
model2.fit(X_train,y_train)
y_train_pred = model2.predict(X_train)
y_val_pred = model2.predict(X_val)
y_test_pred = model2.predict(X_test)

train_rmse2 = mean_squared_error(y_train,y_train_pred,squared=False)
val_rmse2 = mean_squared_error(y_val,y_val_pred,squared=False)

print(f'Train rmse2: {train_rmse2}')
print(f'Val rmse2: {val_rmse2}')

In [ ]:
Y_test = model.predict(X_test).clip(0, 20)
#df_test = pd.read_csv('data/test.csv')
df_test = pd.read_csv('../input/english-converted-datasets/test.csv')  # <-- this is a code fix so the notebook can properly function

submission = pd.DataFrame({
    "ID": df_test.index, 
     "item_cnt_month": Y_test
 })
submission.to_csv('submission.csv', index=False)





# A/B model parameter selection and comparison.

In [ ]:
import xgboost as xgb
ts = time.time()

# First model

model = xgb.XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_val, y_val)], 
    verbose=True, 
    early_stopping_rounds = 10)

time.time() - ts

In [ ]:
Y_test = model.predict(X_test).clip(0, 20)
df_test = pd.read_csv(TEST_CSV)
submission = pd.DataFrame({
    "ID": df_test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('submission_predict_1.csv', index=False)



In [ ]:
import xgboost as xgb2
ts2 = time.time()

# Second model

model2 = xgb2.XGBRegressor(
    max_depth=10,
    n_estimators=1500,
    min_child_weight=360, 
    colsample_bytree=0.9, 
    subsample=0.6, 
    eta=0.2,    
    seed=67)

model2.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_val, y_val)], 
    verbose=True, 
    early_stopping_rounds = 15)

time.time() - ts

In [ ]:
Y_test = model2.predict(X_test).clip(0, 20)
df_test = pd.read_csv(TEST_CSV)
submission = pd.DataFrame({
    "ID": df_test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('submission_predict_2.csv', index=False)

# Any ideas for running a comparison between submission_predict_1 and submission_predict_2 would be appreciated.